---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

def blight_model():
    
    train = pd.read_csv('train.csv', engine='python')
    test = pd.read_csv('test.csv', engine='python')
    X_train = train.merge(pd.read_csv('addresses.csv'), how = 'left',
                          on = ['ticket_id']).merge(pd.read_csv('latlons.csv'), how = 'left', on = ['address'])
    X_test = test.merge(pd.read_csv('addresses.csv'), how = 'left',
                         on = ['ticket_id']).merge(pd.read_csv('latlons.csv'), how = 'left', on = ['address'])
    
    def hs_np(lon2, lat2):
        lon1 =  -83.059
        lat1 = 42.334
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km
    
    X_train = (X_train[(~np.isnan(X_train['compliance']))])

    y_train = X_train['compliance']
    
    X_train['time_to_hearing'] = (pd.to_datetime(X_train['hearing_date'])
                                  - pd.to_datetime(X_train['ticket_issued_date'])).dt.days
    X_train = X_train.drop(['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status',
                           'compliance_detail', 'compliance'], axis = 1)
    feature_NS = ['agency_name', 'violator_name', 'inspector_name', 'violation_street_number', 'violation_street_name',
                  'violation_zip_code', 'violation_description', 'mailing_address_str_number','mailing_address_str_name',
                  'country', 'state', 'zip_code', 'non_us_str_code', 'admin_fee', 'state_fee', 'clean_up_cost',
                  'ticket_issued_date', 'hearing_date', 'grafitti_status', 'ticket_id', 'city', 'violation_code','address',
                  'lat', 'lon']
    
    X_train['dist_to_center'] = hs_np(X_train.ix[:, 'lon'], X_train.ix[:, 'lat']).fillna(0.0)
    X_train = X_train.drop(feature_NS, axis = 1)

    X_test['time_to_hearing'] = (pd.to_datetime(X_test['hearing_date']) - pd.to_datetime(X_test['ticket_issued_date'])).dt.days
    X_test['dist_to_center'] = hs_np(X_test.ix[:, 'lon'], X_test.ix[:, 'lat']).fillna(0.0)
    X_test = X_test.drop(feature_NS, axis = 1)
    
    labels_dispostion = X_train['disposition'].append(X_test['disposition'])
    le = preprocessing.LabelEncoder()
    le.fit_transform(labels_dispostion)
    X_train['disposition'] = le.transform(X_train['disposition'])
    X_test['disposition'] = le.transform(X_test['disposition'])
    
    OHT_enc_train = pd.get_dummies(X_train['disposition'])
    X_train = pd.concat([OHT_enc_train, X_train], axis = 1)
    X_train = X_train.drop(['disposition'], axis = 1)
    OHT_enc_test = pd.get_dummies(X_test['disposition'])
    X_test = pd.concat([OHT_enc_test, X_test], axis = 1)
    X_test = X_test.drop(['disposition'], axis = 1)
    OHTcols_X_test = X_test.columns.tolist()
    X_train = X_train.reindex(columns=OHTcols_X_test)
    X_train.columns = X_train.columns.map(str)
    X_test.columns = X_test.columns.map(str)
    X_train, X_test = X_train.align(X_test, join = 'left', axis = 1)
    
    impute = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
    impute.fit(X_train[['time_to_hearing']])
    X_train['time_to_hearing'] = impute.transform(X_train[['time_to_hearing']])
    X_test['time_to_hearing'] = impute.transform(X_test[['time_to_hearing']])
    X_train = X_train.fillna(False,downcast='infer')
    
    scaler = preprocessing.MinMaxScaler()
    scaler.fit_transform(X_train)
    scaler.transform(X_test) 

    params = {'n_estimators':[50, 100]}
    grad_prams = {'n_estimators': 500, 'min_samples_split':500, 'min_samples_leaf':5, 'max_depth':5, 'random_state': 0,
                  'learning_rate' :0.05}
    rf = GradientBoostingClassifier(**grad_prams)
    rfc = GridSearchCV(rf,  param_grid = params, cv = 5, scoring = 'roc_auc')
    rfc.fit(X_train, y_train)
    pred_prob = rfc.predict_proba(X_test)[:, 1]
    
    results = pd.Series(pred_prob, index = test['ticket_id'], name = 'compliance',  dtype = 'float32')
    
    return results

blight_model()

ticket_id
284932    0.056430
285362    0.021627
285361    0.066240
285338    0.054479
285346    0.066272
285345    0.054172
285347    0.063445
285342    0.858995
285530    0.021627
284989    0.024835
285344    0.084548
285343    0.022242
285340    0.022242
285341    0.063445
285349    0.066272
285348    0.054172
284991    0.024835
285532    0.023390
285406    0.023390
285001    0.035959
285006    0.031653
285405    0.021627
285337    0.023527
285496    0.063445
285497    0.054479
285378    0.021627
285589    0.027796
285585    0.053711
285501    0.071356
285581    0.021627
            ...   
376367    0.023390
376366    0.034253
376362    0.206225
376363    0.206225
376365    0.023390
376364    0.034253
376228    0.075994
376265    0.036333
376286    0.093901
376320    0.040870
376314    0.034450
376327    0.261057
376385    0.228787
376435    0.842893
376370    0.871936
376434    0.063060
376459    0.097434
376478    0.020952
376473    0.038326
376484    0.056479
376482    0.023527
37

In [ ]:
## Your AUC of 0.808319356116 was awarded a value of 1.0 out of 1.0 total grades